PySpark + Spark SQL

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("PracticeProject") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sql("CREATE DATABASE IF NOT EXISTS sales")

customers_data = [
    (101, 'Ali', 'ali@gmail.com', 'Mumbai', '2022-05-10'),
    (102, 'Neha', 'neha@yahoo.com', 'Delhi', '2023-01-15'),
    (103, 'Ravi', 'ravi@hotmail.com', 'Bangalore', '2021-11-01'),
    (104, 'Sneha', 'sneha@outlook.com', 'Hyderabad', '2020-07-22'),
    (105, 'Amit', 'amit@gmail.com', 'Chennai', '2023-03-10'),
]

orders_data = [
    (1, 101, 'Laptop', 'Electronics', 2, 50000.0, '2024-01-10'),
    (2, 101, 'Mouse', 'Electronics', 1, 1200.0, '2024-01-15'),
    (3, 102, 'Tablet', 'Electronics', 1, 20000.0, '2024-02-01'),
    (4, 103, 'Bookshelf', 'Furniture', 1, 3500.0, '2024-02-10'),
    (5, 104, 'Mixer', 'Appliances', 1, 5000.0, '2024-02-15'),
    (6, 105, 'Notebook', 'Stationery', 5, 500.0, '2024-03-01'),
    (7, 102, 'Phone', 'Electronics', 1, 30000.0, '2024-03-02'),
]


customers_df = spark.createDataFrame(customers_data, ["CustomerID", "Name", "Email", "City", "SignupDate"])
orders_df = spark.createDataFrame(orders_data, ["OrderID", "CustomerID", "Product", "Category", "Quantity", "Price", "OrderDate"])


customers_df.write.mode("overwrite").saveAsTable("sales.customers")
orders_df.write.mode("overwrite").saveAsTable("sales.orders")


SECTION A: PySpark DataFrame Tasks

In [3]:
# 1.Add a column
orders_df = orders_df.withColumn("TotalAmount", col("Price") * col("Quantity"))

# 2.Filter
orders_df.filter(col("TotalAmount") > 10000).show()

# 3.Standardize
customers_df = customers_df.withColumn("City", lower(col("City")))

# 4.Extract year
orders_df = orders_df.withColumn("OrderYear", year(to_date("OrderDate")))

# 5.Fill null values
customers_df = customers_df.fillna({'Email': 'unknown@example.com'})

# 6.Use when/otherwise
orders_df = orders_df.withColumn(
    "CategoryType",
    when(col("TotalAmount") < 5000, "Low")
    .when((col("TotalAmount") >= 5000) & (col("TotalAmount") <= 20000), "Medium")
    .otherwise("High")
)
orders_df.show()


+-------+----------+-------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+-------+-----------+--------+-------+----------+-----------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      3|       102| Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      7|       102|  Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+-------+-----------+--------+-------+----------+-----------+

+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+------------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|CategoryType|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+------------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|        High|
|      2

SECTION B: Spark SQL Tasks

In [4]:
# Register as SQL Tables (if not already)
customers_df.createOrReplaceTempView("customers")
orders_df.createOrReplaceTempView("orders")

# 7. Orders by "Ali"
spark.sql("""
SELECT o.* FROM orders o
JOIN customers c ON o.CustomerID = c.CustomerID
WHERE c.Name = 'Ali'
""").show()

# 8. Total Spending by Each Customer
spark.sql("""
SELECT CustomerID, SUM(Price * Quantity) AS TotalSpent
FROM orders
GROUP BY CustomerID
""").show()

# 9. Highest Revenue Category
spark.sql("""
SELECT Category, SUM(Price * Quantity) AS Revenue
FROM orders
GROUP BY Category
ORDER BY Revenue DESC
LIMIT 1
""").show()

# 10. View: CustomerName, Product, TotalAmount
spark.sql("""
CREATE OR REPLACE TEMP VIEW customer_orders AS
SELECT c.Name AS CustomerName, o.Product, o.TotalAmount
FROM orders o
JOIN customers c ON o.CustomerID = c.CustomerID
""")

# 11. Products ordered after Feb 2024
spark.sql("""
SELECT * FROM customer_orders
WHERE TotalAmount IS NOT NULL AND Product IN (
    SELECT Product FROM orders WHERE OrderDate > '2024-02-01'
)
""").show()


+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+------------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|CategoryType|
+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+------------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|        High|
|      2|       101|  Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|         Low|
+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+------------+

+----------+----------+
|CustomerID|TotalSpent|
+----------+----------+
|       101|  101200.0|
|       102|   50000.0|
|       103|    3500.0|
|       104|    5000.0|
|       105|    2500.0|
+----------+----------+

+-----------+--------+
|   Category| Revenue|
+-----------+--------+
|Electronics|151200.0|
+-----------+--------+

+------------+---------+-----------+
|Cust

SECTION C: Advanced Practice

In [5]:
# 12. Global Temp View
customers_df.createOrReplaceGlobalTempView("customers")

# Query it
spark.sql("SELECT * FROM global_temp.customers WHERE City = 'mumbai'").show()

# 13.Save transformed orders_df to Parquet
orders_df.write.mode("overwrite").parquet("/tmp/transformed_orders")

# 14.Read and count
reloaded_df = spark.read.parquet("/tmp/transformed_orders")
print("Order count:", reloaded_df.count())


+----------+----+-------------+------+----------+
|CustomerID|Name|        Email|  City|SignupDate|
+----------+----+-------------+------+----------+
|       101| Ali|ali@gmail.com|mumbai|2022-05-10|
+----------+----+-------------+------+----------+

Order count: 7


 SECTION D: UDF + Built-in Function Tasks

In [6]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# 15.Mask Email UDF
def mask_email(email):
    parts = email.split('@')
    return parts[0][0] + "***@" + parts[1] if len(parts) == 2 else email

mask_udf = udf(mask_email, StringType())
customers_df = customers_df.withColumn("MaskedEmail", mask_udf("Email"))
customers_df.select("Email", "MaskedEmail").show()

# 16.concat_ws - Full Label
customers_df = customers_df.withColumn("FullLabel", concat_ws(" from ", "Name", "City"))
customers_df.select("FullLabel").show()

# 17.Clean product names
orders_df = orders_df.withColumn("CleanProduct", regexp_replace("Product", "[^a-zA-Z0-9]", ""))
orders_df.select("Product", "CleanProduct").show()

# 18.Customer age in days
customers_df = customers_df.withColumn("SignupDate", to_date("SignupDate"))
customers_df = customers_df.withColumn("DaysSinceSignup", datediff(current_date(), col("SignupDate")))
customers_df.select("Name", "DaysSinceSignup").show()


+-----------------+----------------+
|            Email|     MaskedEmail|
+-----------------+----------------+
|    ali@gmail.com|  a***@gmail.com|
|   neha@yahoo.com|  n***@yahoo.com|
| ravi@hotmail.com|r***@hotmail.com|
|sneha@outlook.com|s***@outlook.com|
|   amit@gmail.com|  a***@gmail.com|
+-----------------+----------------+

+--------------------+
|           FullLabel|
+--------------------+
|     Ali from mumbai|
|     Neha from delhi|
| Ravi from bangalore|
|Sneha from hyderabad|
|   Amit from chennai|
+--------------------+

+---------+------------+
|  Product|CleanProduct|
+---------+------------+
|   Laptop|      Laptop|
|    Mouse|       Mouse|
|   Tablet|      Tablet|
|Bookshelf|   Bookshelf|
|    Mixer|       Mixer|
| Notebook|    Notebook|
|    Phone|       Phone|
+---------+------------+

+-----+---------------+
| Name|DaysSinceSignup|
+-----+---------------+
|  Ali|           1121|
| Neha|            871|
| Ravi|           1311|
|Sneha|           1778|
| Amit|       